In [93]:
import tensorflow as tf
import pandas as pd
import sys
sys.path.append('../')
from config import *
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.preprocessing import sequence
import numpy as np
from tqdm import tqdm
from IPython.display import clear_output

In [94]:
import warnings
warnings.filterwarnings("ignore")

In [95]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [96]:
tf.enable_eager_execution()
tf.test.is_gpu_available()

True

In [ ]:
### 占用gpu
a = tf.constant(1)
res = tf.add(a, a)
sess = tf.Session()
sess.run(res)

In [97]:
### Hyperparameter
###################
SAVE_PATH = '/home/huangzc/competition/tencent/model_ckpt/simple_nn/model.ckpt'
EMB_SIZE = 50
BATCH_SIZE = 10240
EPOCHS = 100

In [ ]:
tr_log = pd.read_pickle(TRAIN_DIR+USER_LOG_PATH)
ts_log = pd.read_pickle(TEST_DIR+USER_LOG_PATH)

tr_ad_df = pd.read_pickle(TRAIN_DIR+AD_INFO_PATH)
ts_ad_df = pd.read_pickle(TEST_DIR+AD_INFO_PATH)

assert tr_log['creative_id'].values.tolist() == tr_ad_df['creative_id'].values.tolist()
assert ts_log['creative_id'].values.tolist() == ts_ad_df['creative_id'].values.tolist()

tr_ad_id_log = pd.concat([tr_log, tr_ad_df], axis=1)
ts_ad_id_log = pd.concat([ts_log, ts_ad_df], axis=1)

tr_ad_id_log = tr_ad_id_log[['user_id', 'age', 'gender', 'ad_id']]
ts_ad_id_log = ts_ad_id_log[['user_id', 'ad_id']]

In [141]:
#读取
f = open(TRAIN_DIR+'gensim_ad_id_dict.js','r')
a = f.read()
vocab_dict = eval(a)
f.close()
wv_matrix = np.load(TRAIN_DIR+'gensim_ad_id.npy')
vocab = vocab_dict.keys()
vocab_size = len(vocab)

In [99]:
tr_ad_id_log['ad_id'] = tr_ad_id_log['ad_id'].astype(str)
ts_ad_id_log['ad_id'] = ts_ad_id_log['ad_id'].astype(str)

tr_ad_id_log = tr_ad_id_log[tr_ad_id_log['ad_id'].isin(vocab)]
ts_ad_id_log = ts_ad_id_log[ts_ad_id_log['ad_id'].isin(vocab)]

In [100]:
tr_ad_id_log['ad_id_index'] = tr_ad_id_log['ad_id'].map(lambda x: vocab_dict[x])
ts_ad_id_log['ad_id_index'] = ts_ad_id_log['ad_id'].map(lambda x: vocab_dict[x])

In [101]:
def get_clk_list(df):
    return list(df.values)

In [102]:
grid_df = tr_ad_id_log.groupby(['user_id']).agg({'age': 'first', 'gender': 'first', 'ad_id_index': lambda x: get_clk_list(x)}).reset_index()
grid_df_ts = ts_ad_id_log.groupby(['user_id']).agg({'ad_id_index': lambda x: get_clk_list(x)}).reset_index()

In [103]:
feature_columns = ['ad_id_index']
TARGET = 'age'

grid_df[TARGET] = grid_df[TARGET] - 1

In [104]:
### Split into train and valid dataset
train = grid_df[feature_columns+[TARGET]]
test = grid_df_ts[feature_columns]

train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

### x, y dataframe
x_train, y_train = train[feature_columns], train[TARGET]
x_val, y_val = val[feature_columns], val[TARGET]
x_test = test

720000 train examples
180000 validation examples
1000000 test examples


In [105]:
sentence_size = int(grid_df[feature_columns[0]].map(lambda x: len(x)).quantile(0.9))
print('choose sentences max len: %d' % sentence_size)

choose sentences max len: 65


In [106]:
print("Pad sequences (samples x time)")

### feature_columns[0] means click list
x_train = sequence.pad_sequences(x_train[feature_columns[0]],
                                 maxlen=sentence_size, 
                                 padding='post', 
                                 truncating='post',
                                 dtype='int64',
                                 value=float(vocab_size)
                                 )
x_val = sequence.pad_sequences(x_val[feature_columns[0]],
                                 maxlen=sentence_size, 
                                 padding='post', 
                                 truncating='post',
                                 dtype='int64',
                                 value=float(vocab_size)
                                 )
x_test = sequence.pad_sequences(x_test[feature_columns[0]], 
                                maxlen=sentence_size, 
                                padding='post',
                                truncating='post',
                                dtype='int64',
                                value=float(vocab_size)
                               )
print("x_train shape:", x_train.shape)
print("x_val shape:", x_val.shape)
print("x_test shape:", x_test.shape)

Pad sequences (samples x time)
x_train shape: (720000, 65)
x_val shape: (180000, 65)
x_test shape: (1000000, 65)


In [107]:
### Get tf dataset
def get_train_ds(x, y): 
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(buffer_size=len(x))
    dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.repeat(EPOCHS)
    return dataset

def get_test_ds(x, ): 
    dataset = tf.data.Dataset.from_tensor_slices((x, )).batch(BATCH_SIZE)
    return dataset

train_ds = get_train_ds(x_train, y_train.values)
valid_ds = get_train_ds(x_val, y_val.values)
test_ds = get_test_ds(x_test)

In [142]:
weights = np.random.rand(1, wv_matrix.shape[1])
weights = np.concatenate([wv_matrix, weights], axis=0)

In [143]:
### Construct Model
#################################
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size+1, EMB_SIZE, weights=[weights])
#         self.embedding = tf.keras.layers.Embedding(vocab_size, EMB_SIZE)
        self.pool1D = tf.keras.layers.GlobalAveragePooling1D()
        self.dense1 = tf.keras.layers.Dense(16, activation=tf.nn.leaky_relu)
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.pool1D(x)
        x = self.dense1(x)
        return self.dense2(x)

model = MyModel()

In [136]:
# Restore the weights
# model.load_weights(SAVE_PATH)

In [144]:
### Loss & Metric
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [145]:
@tf.function
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [146]:
@tf.function
def valid_step(features, labels):
    predictions = model(features)
    v_loss = loss_object(labels, predictions)

    valid_loss(v_loss)
    valid_accuracy(labels, predictions)

In [147]:
for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    valid_loss.reset_states()
    valid_accuracy.reset_states()

    CNT = 0
    for features, labels in train_ds:
#         clear_output(wait=True)
        train_step(features, labels)
        CNT += 1
        print('batch: %d, batch loss: %.4f' % (CNT, train_loss.result()))    
        
    for val_features, val_labels in valid_ds:
        valid_step(val_features, val_labels)

    template = 'Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}%, Valid Loss: {:.2f}, Valid Accuracy: {:.2f}%'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         valid_loss.result(),
                         valid_accuracy.result()*100))

batch: 1, batch loss: 2.4333
batch: 2, batch loss: 2.4251
batch: 3, batch loss: 2.4207
batch: 4, batch loss: 2.4132
batch: 5, batch loss: 2.4082
batch: 6, batch loss: 2.4037
batch: 7, batch loss: 2.3981
batch: 8, batch loss: 2.3928
batch: 9, batch loss: 2.3876
batch: 10, batch loss: 2.3830
batch: 11, batch loss: 2.3789
batch: 12, batch loss: 2.3742
batch: 13, batch loss: 2.3700
batch: 14, batch loss: 2.3653
batch: 15, batch loss: 2.3610
batch: 16, batch loss: 2.3569
batch: 17, batch loss: 2.3525
batch: 18, batch loss: 2.3483
batch: 19, batch loss: 2.3439
batch: 20, batch loss: 2.3395
batch: 21, batch loss: 2.3354
batch: 22, batch loss: 2.3310
batch: 23, batch loss: 2.3270
batch: 24, batch loss: 2.3229
batch: 25, batch loss: 2.3193
batch: 26, batch loss: 2.3153
batch: 27, batch loss: 2.3116
batch: 28, batch loss: 2.3077
batch: 29, batch loss: 2.3037
batch: 30, batch loss: 2.2996
batch: 31, batch loss: 2.2960
batch: 32, batch loss: 2.2922
batch: 33, batch loss: 2.2884
batch: 34, batch lo

batch: 55, batch loss: 1.6769
batch: 56, batch loss: 1.6764
batch: 57, batch loss: 1.6759
batch: 58, batch loss: 1.6754
batch: 59, batch loss: 1.6749
batch: 60, batch loss: 1.6744
batch: 61, batch loss: 1.6738
batch: 62, batch loss: 1.6734
batch: 63, batch loss: 1.6728
batch: 64, batch loss: 1.6721
batch: 65, batch loss: 1.6714
batch: 66, batch loss: 1.6712
batch: 67, batch loss: 1.6707
batch: 68, batch loss: 1.6701
batch: 69, batch loss: 1.6696
batch: 70, batch loss: 1.6692
batch: 71, batch loss: 1.6686
Epoch 4, Loss: 1.67, Accuracy: 33.21%, Valid Loss: 1.63, Valid Accuracy: 34.24%
batch: 1, batch loss: 1.6360
batch: 2, batch loss: 1.6280
batch: 3, batch loss: 1.6215
batch: 4, batch loss: 1.6223
batch: 5, batch loss: 1.6230
batch: 6, batch loss: 1.6233
batch: 7, batch loss: 1.6230
batch: 8, batch loss: 1.6220
batch: 9, batch loss: 1.6209
batch: 10, batch loss: 1.6221
batch: 11, batch loss: 1.6223
batch: 12, batch loss: 1.6219
batch: 13, batch loss: 1.6221
batch: 14, batch loss: 1.6212

batch: 35, batch loss: 1.5184
batch: 36, batch loss: 1.5183
batch: 37, batch loss: 1.5187
batch: 38, batch loss: 1.5188
batch: 39, batch loss: 1.5185
batch: 40, batch loss: 1.5184
batch: 41, batch loss: 1.5183
batch: 42, batch loss: 1.5183
batch: 43, batch loss: 1.5180
batch: 44, batch loss: 1.5178
batch: 45, batch loss: 1.5176
batch: 46, batch loss: 1.5176
batch: 47, batch loss: 1.5175
batch: 48, batch loss: 1.5172
batch: 49, batch loss: 1.5171
batch: 50, batch loss: 1.5168
batch: 51, batch loss: 1.5168
batch: 52, batch loss: 1.5169
batch: 53, batch loss: 1.5167
batch: 54, batch loss: 1.5165
batch: 55, batch loss: 1.5165
batch: 56, batch loss: 1.5165
batch: 57, batch loss: 1.5164
batch: 58, batch loss: 1.5164
batch: 59, batch loss: 1.5164
batch: 60, batch loss: 1.5164
batch: 61, batch loss: 1.5161
batch: 62, batch loss: 1.5161
batch: 63, batch loss: 1.5160
batch: 64, batch loss: 1.5157
batch: 65, batch loss: 1.5155
batch: 66, batch loss: 1.5158
batch: 67, batch loss: 1.5157
batch: 68,

batch: 15, batch loss: 1.4572
batch: 16, batch loss: 1.4574
batch: 17, batch loss: 1.4570
batch: 18, batch loss: 1.4574
batch: 19, batch loss: 1.4571
batch: 20, batch loss: 1.4576
batch: 21, batch loss: 1.4578
batch: 22, batch loss: 1.4579
batch: 23, batch loss: 1.4576
batch: 24, batch loss: 1.4576
batch: 25, batch loss: 1.4577
batch: 26, batch loss: 1.4581
batch: 27, batch loss: 1.4587
batch: 28, batch loss: 1.4583
batch: 29, batch loss: 1.4583
batch: 30, batch loss: 1.4584
batch: 31, batch loss: 1.4585
batch: 32, batch loss: 1.4582
batch: 33, batch loss: 1.4583
batch: 34, batch loss: 1.4583
batch: 35, batch loss: 1.4581
batch: 36, batch loss: 1.4581
batch: 37, batch loss: 1.4587
batch: 38, batch loss: 1.4588
batch: 39, batch loss: 1.4586
batch: 40, batch loss: 1.4585
batch: 41, batch loss: 1.4584
batch: 42, batch loss: 1.4585
batch: 43, batch loss: 1.4583
batch: 44, batch loss: 1.4582
batch: 45, batch loss: 1.4579
batch: 46, batch loss: 1.4579
batch: 47, batch loss: 1.4578
batch: 48,

batch: 68, batch loss: 1.4213
batch: 69, batch loss: 1.4212
batch: 70, batch loss: 1.4212
batch: 71, batch loss: 1.4199
Epoch 15, Loss: 1.42, Accuracy: 41.87%, Valid Loss: 1.48, Valid Accuracy: 39.53%
batch: 1, batch loss: 1.4199
batch: 2, batch loss: 1.4131
batch: 3, batch loss: 1.4068
batch: 4, batch loss: 1.4072
batch: 5, batch loss: 1.4079
batch: 6, batch loss: 1.4076
batch: 7, batch loss: 1.4072
batch: 8, batch loss: 1.4069
batch: 9, batch loss: 1.4056
batch: 10, batch loss: 1.4074
batch: 11, batch loss: 1.4081
batch: 12, batch loss: 1.4083
batch: 13, batch loss: 1.4085
batch: 14, batch loss: 1.4086
batch: 15, batch loss: 1.4093
batch: 16, batch loss: 1.4095
batch: 17, batch loss: 1.4091
batch: 18, batch loss: 1.4096
batch: 19, batch loss: 1.4094
batch: 20, batch loss: 1.4097
batch: 21, batch loss: 1.4100
batch: 22, batch loss: 1.4099
batch: 23, batch loss: 1.4098
batch: 24, batch loss: 1.4098
batch: 25, batch loss: 1.4099
batch: 26, batch loss: 1.4103
batch: 27, batch loss: 1.410

...

batch: 1, batch loss: 0.3658
batch: 2, batch loss: 0.3618
batch: 3, batch loss: 0.3635
batch: 4, batch loss: 0.3660
batch: 5, batch loss: 0.3662
batch: 6, batch loss: 0.3659
batch: 7, batch loss: 0.3650
batch: 8, batch loss: 0.3651
batch: 9, batch loss: 0.3645
batch: 10, batch loss: 0.3650
batch: 11, batch loss: 0.3653
batch: 12, batch loss: 0.3653
batch: 13, batch loss: 0.3650
batch: 14, batch loss: 0.3647
batch: 15, batch loss: 0.3652
batch: 16, batch loss: 0.3661
batch: 17, batch loss: 0.3652
batch: 18, batch loss: 0.3651
batch: 19, batch loss: 0.3647
batch: 20, batch loss: 0.3643
batch: 21, batch loss: 0.3645
batch: 22, batch loss: 0.3647
batch: 23, batch loss: 0.3653
batch: 24, batch loss: 0.3652
batch: 25, batch loss: 0.3650
batch: 26, batch loss: 0.3650
batch: 27, batch loss: 0.3649
batch: 28, batch loss: 0.3645
batch: 29, batch loss: 0.3647
batch: 30, batch loss: 0.3646
batch: 31, batch loss: 0.3640
batch: 32, batch loss: 0.3637
batch: 33, batch loss: 0.3635
batch: 34, batch lo

batch: 55, batch loss: 0.3327
batch: 56, batch loss: 0.3327
batch: 57, batch loss: 0.3327
batch: 58, batch loss: 0.3327
batch: 59, batch loss: 0.3325
batch: 60, batch loss: 0.3325
batch: 61, batch loss: 0.3325
batch: 62, batch loss: 0.3325
batch: 63, batch loss: 0.3324
batch: 64, batch loss: 0.3323
batch: 65, batch loss: 0.3322
batch: 66, batch loss: 0.3323
batch: 67, batch loss: 0.3322
batch: 68, batch loss: 0.3323
batch: 69, batch loss: 0.3322
batch: 70, batch loss: 0.3322
batch: 71, batch loss: 0.3296
Epoch 86, Loss: 0.33, Accuracy: 91.14%, Valid Loss: 3.78, Valid Accuracy: 36.65%
batch: 1, batch loss: 0.3189
batch: 2, batch loss: 0.3183
batch: 3, batch loss: 0.3222
batch: 4, batch loss: 0.3268
batch: 5, batch loss: 0.3266
batch: 6, batch loss: 0.3257
batch: 7, batch loss: 0.3244
batch: 8, batch loss: 0.3247
batch: 9, batch loss: 0.3245
batch: 10, batch loss: 0.3249
batch: 11, batch loss: 0.3250
batch: 12, batch loss: 0.3245
batch: 13, batch loss: 0.3242
batch: 14, batch loss: 0.323

batch: 35, batch loss: 0.2929
batch: 36, batch loss: 0.2927
batch: 37, batch loss: 0.2926
batch: 38, batch loss: 0.2926
batch: 39, batch loss: 0.2929
batch: 40, batch loss: 0.2927
batch: 41, batch loss: 0.2928
batch: 42, batch loss: 0.2927
batch: 43, batch loss: 0.2925
batch: 44, batch loss: 0.2925
batch: 45, batch loss: 0.2923
batch: 46, batch loss: 0.2923
batch: 47, batch loss: 0.2922
batch: 48, batch loss: 0.2920
batch: 49, batch loss: 0.2921
batch: 50, batch loss: 0.2921
batch: 51, batch loss: 0.2921
batch: 52, batch loss: 0.2921
batch: 53, batch loss: 0.2920
batch: 54, batch loss: 0.2919
batch: 55, batch loss: 0.2918
batch: 56, batch loss: 0.2918
batch: 57, batch loss: 0.2917
batch: 58, batch loss: 0.2918
batch: 59, batch loss: 0.2916
batch: 60, batch loss: 0.2917
batch: 61, batch loss: 0.2918
batch: 62, batch loss: 0.2918
batch: 63, batch loss: 0.2916
batch: 64, batch loss: 0.2915
batch: 65, batch loss: 0.2915
batch: 66, batch loss: 0.2916
batch: 67, batch loss: 0.2916
batch: 68,

batch: 15, batch loss: 0.2627
batch: 16, batch loss: 0.2633
batch: 17, batch loss: 0.2623
batch: 18, batch loss: 0.2620
batch: 19, batch loss: 0.2620
batch: 20, batch loss: 0.2619
batch: 21, batch loss: 0.2622
batch: 22, batch loss: 0.2621
batch: 23, batch loss: 0.2624
batch: 24, batch loss: 0.2623
batch: 25, batch loss: 0.2622
batch: 26, batch loss: 0.2623
batch: 27, batch loss: 0.2621
batch: 28, batch loss: 0.2618
batch: 29, batch loss: 0.2619
batch: 30, batch loss: 0.2617
batch: 31, batch loss: 0.2613
batch: 32, batch loss: 0.2611
batch: 33, batch loss: 0.2609
batch: 34, batch loss: 0.2608
batch: 35, batch loss: 0.2607
batch: 36, batch loss: 0.2605
batch: 37, batch loss: 0.2604
batch: 38, batch loss: 0.2604
batch: 39, batch loss: 0.2606
batch: 40, batch loss: 0.2605
batch: 41, batch loss: 0.2606
batch: 42, batch loss: 0.2605
batch: 43, batch loss: 0.2604
batch: 44, batch loss: 0.2603
batch: 45, batch loss: 0.2601
batch: 46, batch loss: 0.2601
batch: 47, batch loss: 0.2600
batch: 48,

batch: 68, batch loss: 0.2386
batch: 69, batch loss: 0.2385
batch: 70, batch loss: 0.2385
batch: 71, batch loss: 0.2366
Epoch 97, Loss: 0.24, Accuracy: 94.43%, Valid Loss: 4.52, Valid Accuracy: 36.22%
batch: 1, batch loss: 0.2329
batch: 2, batch loss: 0.2325
batch: 3, batch loss: 0.2334
batch: 4, batch loss: 0.2355
batch: 5, batch loss: 0.2349
batch: 6, batch loss: 0.2350
batch: 7, batch loss: 0.2349
batch: 8, batch loss: 0.2353
batch: 9, batch loss: 0.2348
batch: 10, batch loss: 0.2349
batch: 11, batch loss: 0.2348
batch: 12, batch loss: 0.2346
batch: 13, batch loss: 0.2348
batch: 14, batch loss: 0.2349
batch: 15, batch loss: 0.2354
batch: 16, batch loss: 0.2362
batch: 17, batch loss: 0.2352
batch: 18, batch loss: 0.2348
batch: 19, batch loss: 0.2346
batch: 20, batch loss: 0.2344
batch: 21, batch loss: 0.2347
batch: 22, batch loss: 0.2348
batch: 23, batch loss: 0.2352
batch: 24, batch loss: 0.2352
batch: 25, batch loss: 0.2350
batch: 26, batch loss: 0.2350
batch: 27, batch loss: 0.234

win:10, 39.78 <br>
win: 2, 39.38 <br>
win:15, 40.18 <br>
win:20, 

In [ ]:
# 保存权重
model.save_weights(SAVE_PATH)